# Import statements:

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.preprocessing import StandardScaler


Bad key "gaxes.labelsize" on line 1 in
/Users/priya/.matplotlib/stylelib/presentation.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
np.random.seed(101)



# Loading Data

In [3]:
train = pd.read_csv('train.csv')

## X = train[['Bedroom AbvGr', 'Full Bath', 'Half Bath',
          ## 'Garage Cars', 'Year Built', 'Year Remod/Add']].copy()
X = train.select_dtypes(include=[np.number]).fillna(0).drop('SalePrice', axis=1)
y = train['SalePrice']

features = X.columns

# Cleaning X 

In [4]:
X.head()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold
0,109,533352170,60,0.0,13517,6,8,1976,2005,289.0,...,475.0,0,44,0,0,0,0,0,3,2010
1,544,531379050,60,43.0,11492,7,5,1996,1997,132.0,...,559.0,0,74,0,0,0,0,0,4,2009
2,153,535304180,20,68.0,7922,5,7,1953,2007,0.0,...,246.0,0,52,0,0,0,0,0,1,2010
3,318,916386060,60,73.0,9802,5,5,2006,2007,0.0,...,400.0,100,0,0,0,0,0,0,4,2010
4,255,906425045,50,82.0,14235,6,8,1900,1993,0.0,...,484.0,0,59,0,0,0,0,0,3,2010


In [5]:
X.columns

Index(['Id', 'PID', 'MS SubClass', 'Lot Frontage', 'Lot Area', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Mas Vnr Area',
       'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area',
       'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath',
       'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Fireplaces',
       'Garage Yr Blt', 'Garage Cars', 'Garage Area', 'Wood Deck SF',
       'Open Porch SF', 'Enclosed Porch', '3Ssn Porch', 'Screen Porch',
       'Pool Area', 'Misc Val', 'Mo Sold', 'Yr Sold'],
      dtype='object')

In [6]:
X.isnull().sum()

Id                 0
PID                0
MS SubClass        0
Lot Frontage       0
Lot Area           0
Overall Qual       0
Overall Cond       0
Year Built         0
Year Remod/Add     0
Mas Vnr Area       0
BsmtFin SF 1       0
BsmtFin SF 2       0
Bsmt Unf SF        0
Total Bsmt SF      0
1st Flr SF         0
2nd Flr SF         0
Low Qual Fin SF    0
Gr Liv Area        0
Bsmt Full Bath     0
Bsmt Half Bath     0
Full Bath          0
Half Bath          0
Bedroom AbvGr      0
Kitchen AbvGr      0
TotRms AbvGrd      0
Fireplaces         0
Garage Yr Blt      0
Garage Cars        0
Garage Area        0
Wood Deck SF       0
Open Porch SF      0
Enclosed Porch     0
3Ssn Porch         0
Screen Porch       0
Pool Area          0
Misc Val           0
Mo Sold            0
Yr Sold            0
dtype: int64

In [7]:
X['Bedroom AbvGr'].isnull().sum()

0

In [8]:
X[['Garage Cars']].isnull().sum()

Garage Cars    0
dtype: int64

In [9]:
X[['Year Remod/Add']].isnull().sum()

Year Remod/Add    0
dtype: int64

In [10]:
X['Garage Cars'].value_counts(dropna=False)

2.0    1136
1.0     524
3.0     263
0.0     114
4.0      13
5.0       1
Name: Garage Cars, dtype: int64

In [11]:
X.fillna({'Garage Cars':0}, inplace=True)

# Train / Test Split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [13]:
ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

In [14]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1538, 38) (1538,)
(513, 38) (513,)


# Modeling

## BASIC LINEAR REGRESSION

In [15]:
lm = LinearRegression()
lm.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [16]:
lm.score(X_train, y_train)

0.8365880212145547

In [17]:
cross_val_score(lm, X_train, y_train, cv=3).mean()


0.77704376997969

## BASIC LASSO REGRESSION

In [18]:
la = LassoCV(cv=3)
la.fit(X_train, y_train)

LassoCV(alphas=None, copy_X=True, cv=3, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [19]:
la.alpha_

1167.0050470298615

In [20]:
la.score(X_train, y_train)

0.8306701186327432

In [21]:
cross_val_score(la, X_train, y_train, cv=3).mean()


0.7665879775444632

## FINAL MODEL EVALUATION

In [22]:
# We want to test out whether linear regression or lasso is better
lm.score(X_test, y_test)

0.8285129126742327

In [23]:
la.score(X_test, y_test)

0.8354641299031973

In [24]:
## Lasso Regression model is better

## SUBMISSION


In [25]:
test = pd.read_csv('test.csv')

## X = test[['Bedroom AbvGr', 'Full Bath', 'Half Bath',
          ## 'Garage Cars', 'Year Built', 'Year Remod/Add']].copy()
X_eval = test.select_dtypes(include=[np.number]).fillna(0)

In [26]:
X_eval.head()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold
0,2658,902301120,190,69.0,9142,6,8,1910,1950,0.0,...,440.0,0,60,112,0,0,0,0,4,2006
1,2718,905108090,90,0.0,9662,5,4,1977,1977,0.0,...,580.0,170,0,0,0,0,0,0,8,2006
2,2414,528218130,60,58.0,17104,7,5,2006,2006,0.0,...,426.0,100,24,0,0,0,0,0,9,2006
3,1989,902207150,30,60.0,8520,5,6,1923,2006,0.0,...,480.0,0,0,184,0,0,0,0,7,2007
4,625,535105100,20,0.0,9500,6,5,1963,1963,247.0,...,514.0,0,76,0,0,185,0,0,7,2009


In [27]:
X_eval_t = pd.DataFrame(ss.transform(X_eval), columns=features)

In [28]:
X_eval_t.shape

(879, 38)

In [29]:
X_eval_t

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold
0,1.407389,0.980847,3.117343,0.314010,-0.138365,-0.085858,2.212169,-2.033968,-1.622513,-0.552842,...,-0.143504,-0.718561,0.189149,1.491990,-0.108611,-0.287950,-0.066594,-0.076562,-0.785921,-1.354671
1,1.478449,0.995717,0.777834,-1.763508,-0.067134,-0.799639,-1.432519,0.181829,-0.338760,-0.552842,...,0.512406,0.616404,-0.710065,-0.375285,-0.108611,-0.287950,-0.066594,-0.076562,0.672597,-1.354671
2,1.118410,-1.000929,0.075981,-0.017188,0.952298,0.627923,-0.521347,1.140906,1.040085,-0.552842,...,-0.209095,0.066713,-0.350379,-0.375285,-0.108611,-0.287950,-0.066594,-0.076562,1.037226,-1.354671
3,0.615064,0.980349,-0.625872,0.043030,-0.223569,-0.799639,0.389825,-1.604037,1.040085,-0.552842,...,0.043899,-0.718561,-0.710065,2.692381,-0.108611,-0.287950,-0.066594,-0.076562,0.307967,-0.596765
4,-1.000377,-0.964444,-0.859823,-1.763508,-0.089325,-0.085858,-0.521347,-0.281173,-1.004410,0.846189,...,0.203191,-0.718561,0.428939,-0.375285,-0.108611,2.970882,-0.066594,-0.076562,0.307967,0.919047
5,-1.346205,1.091713,2.415490,-1.131220,-1.131770,-1.513419,0.389825,0.016471,-0.576492,-0.552842,...,-0.865005,-0.718561,-0.710065,0.691730,-0.108611,-0.287950,-0.066594,-0.076562,-0.056662,1.676954
6,-0.168970,0.981514,-0.859823,-0.197842,-0.224117,-1.513419,0.389825,-0.446531,1.040085,-0.552842,...,-0.761934,-0.718561,-0.710065,-0.375285,-0.108611,-0.287950,-0.066594,-0.076562,-0.421292,0.161141
7,-0.724426,1.006811,-0.859823,-1.763508,-0.118639,-0.799639,1.300997,0.181829,0.231796,-0.552842,...,-1.024298,0.639962,-0.710065,-0.375285,-0.108611,-0.287950,-0.066594,-0.076562,1.401855,0.919047
8,-1.628078,-0.974494,2.415490,-0.589259,-0.909172,0.627923,-0.521347,1.074762,0.944993,-0.552842,...,0.549887,-0.718561,0.953480,-0.375285,-0.108611,-0.287950,-0.066594,-0.076562,-1.879809,1.676954
9,0.116457,1.045346,-0.859823,0.494664,-0.003710,-0.085858,0.389825,0.181829,-0.338760,-0.552842,...,0.011103,1.150390,0.533847,-0.375285,-0.108611,-0.287950,-0.066594,-0.076562,-1.515180,0.161141


In [30]:
predicts = lm.predict(X_eval_t)

In [31]:
pd.read_csv('sample_sub_reg.csv').head(3)

,Id,SalePrice
0,2,181479.1217
1,4,181479.1217
2,6,181479.1217


In [32]:
submission = pd.DataFrame({'Id':X_eval['Id'],'SalePrice':predicts})
submission.to_csv('Priya_submission_1.csv', index=False) 

In [33]:
submission.head()

,Id,SalePrice
0,2658,126352.250257
1,2718,151719.958629
2,2414,217429.653708
3,1989,117117.245129
4,625,196835.506957
